<a href="https://colab.research.google.com/github/gcerbaro/DSFinal_global_cancer_patients/blob/main/src/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/gcerbaro/DSFinal_global_cancer_patients.git
#%cd DSFinal_global_cancer_patients

Cloning into 'DSFinal_global_cancer_patients'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15 (delta 2), reused 10 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 1.19 MiB | 7.35 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/DSFinal_global_cancer_patients


In [ ]:
import pandas as pd
import matplotlib
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
#!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 906 bytes | 906.00 KiB/s, done.
From https://github.com/gcerbaro/DSFinal_global_cancer_patients
   4f2d2a2..4e70838  main       -> origin/main
Updating 4f2d2a2..4e70838
Fast-forward
 README.md | 29 ++++++++++++++++++++++++++++-
 1 file changed, 28 insertions(+), 1 deletion(-)


In [10]:
from sklearn.model_selection import train_test_split

data = pd.read_csv("./dataset/global_cancer_patients_2015_2024.csv")
data.head(5)

,Patient_ID,Age,Gender,Country_Region,Year,Genetic_Risk,Air_Pollution,Alcohol_Use,Smoking,Obesity_Level,Cancer_Type,Cancer_Stage,Treatment_Cost_USD,Survival_Years,Target_Severity_Score
0,PT0000000,71,Male,UK,2021,6.4,2.8,9.5,0.9,8.7,Lung,Stage III,62913.44,5.9,4.92
1,PT0000001,34,Male,China,2021,1.3,4.5,3.7,3.9,6.3,Leukemia,Stage 0,12573.41,4.7,4.65
2,PT0000002,80,Male,Pakistan,2023,7.4,7.9,2.4,4.7,0.1,Breast,Stage II,6984.33,7.1,5.84
3,PT0000003,40,Male,UK,2015,1.7,2.9,4.8,3.5,2.7,Colon,Stage I,67446.25,1.6,3.12
4,PT0000004,43,Female,Brazil,2017,5.1,2.8,2.3,6.7,0.5,Skin,Stage III,77977.12,2.9,3.62
